Prerequisite packages

In [ ]:
import os,shutil,pickle,tqdm,sys,random,re,string,pause, datetime,glob
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# # The GPU id to use, usually either "0" or "1"
# os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import keras
import sent2vec
import seq2seq
from seq2seq.models import AttentionSeq2Seq
from seq2seq.models import Seq2Seq

import tensorflow as tf
import tensorboard as tb

import numpy as np
import pandas as pd
import tqdm
from tqdm import tqdm
from random import shuffle
from math import log, floor

from keras.utils import multi_gpu_model

# from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
from keras.metrics import *

import gensim
from gensim.models.word2vec import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *

from sklearn.model_selection import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn.metrics.pairwise import *
from sklearn.neighbors import KNeighborsClassifier
# from collections import Counter
from keras.utils.generic_utils import *
from keras import regularizers
import unicodedata as udata
from keras.applications import *
from keras.preprocessing.image import *

from keras import backend 
from imblearn.ensemble import *
from imblearn.combine import *
# from python.keras import backend 
# Embedding(10,20)
from keras_transformer.extras import ReusableEmbedding, TiedOutputEmbedding
from keras_transformer.position import TransformerCoordinateEmbedding
from keras_transformer.transformer import TransformerACT, TransformerBlock
from keras_transformer.bert import (
    BatchGeneratorForBERT, masked_perplexity,
    MaskedPenalizedSparseCategoricalCrossentropy)

import keras_metrics as km
from keras_trans_mask import RemoveMask, RestoreMask

from keras_multi_head import *
from keras_self_attention import SeqSelfAttention

from sklearn.metrics.pairwise import *
from sklearn.model_selection import *
from sklearn.manifold import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn import preprocessing

from sklearn.metrics import *
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter

In [ ]:
#neede files in same dir.
from models import transformer_bert_model
from bpe import BPEEncoder
from utils import *

load data

In [ ]:
root_dir = './data/tree-rep-profiles_o2o/'
train_pid_paths = pickle.load(open(root_dir + 'train_pid_paths.pkl','rb'))
valid_pid_paths = pickle.load(open(root_dir + 'valid_pid_paths.pkl','rb'))
test_pid_paths = pickle.load(open(root_dir + 'test_pid_paths.pkl','rb'))
exp_pid_paths = pickle.load(open(root_dir + 'exp_pid_paths.pkl','rb'))
# o2o_pid_paths = pickle.load(open(root_dir + 'o2o_pid_paths.pkl','rb'))

train_emb_api,train_emb , train_rep_ans = pickle.load(open(root_dir + 'pids_train.pkl','rb'))
valid_emb_api,valid_emb, valid_rep_ans = pickle.load(open(root_dir + 'pids_valid.pkl','rb'))
test_emb_api,test_emb ,test_rep_ans = pickle.load(open(root_dir + 'pids_test.pkl','rb'))
# o2o_api,o2o_emb,o2o_rep = pickle.load(open(root_dir + 'pids_o2o.pkl','rb'))
exp_api,exp_emb = pickle.load(open(root_dir + 'pids_exp.pkl','rb'))
emb_matrix = pickle.load(open('data/api_emb_matrix.pkl','rb'))

In [ ]:
def basic_statistics(all_length):
    '''
    input: length list of elements
    output1: mean、std、mode、min、q1、median(q2)、q3、max、iqr、outlier、far out
    output2: statistics graph、10%~90% form
    '''
    stat_dict = {}
    stat_dict['mean'] = np.mean(all_length)
    stat_dict['std'] = np.std(all_length)
    stat_dict['mode'] = np.argmax(np.bincount(all_length))
    stat_dict['min'] = np.min(all_length)
    stat_dict['q1'] = np.quantile(all_length,0.25)
    stat_dict['median'] = np.quantile(all_length,0.5)
    stat_dict['q3'] = np.quantile(all_length,0.75)
    stat_dict['max'] = np.max(all_length)
    stat_dict['iqr'] = stat_dict['q3'] - stat_dict['q1']
    stat_dict['outlier'] = stat_dict['q3'] + 1.5*stat_dict['iqr']
    stat_dict['far_out'] = stat_dict['q3'] + 3*stat_dict['iqr']
    for i in [10,20,30,40,50,60,70,80,90,100]:
        stat_dict[str(i)+'%'] = np.percentile(all_length,i)
    return pd.DataFrame.from_dict(stat_dict,orient='index',columns=['length'])

load trained NN model

In [ ]:
def bin_acc(y_true, y_pred):
    return binary_accuracy(y_true, y_pred)

In [ ]:
model_save_path = './model/o2o_stage_gru_selfatt/XXX.h5' #訓練好的模型
model = load_model(model_save_path+'_all.h5',custom_objects={'bin_acc': bin_acc, 'SeqSelfAttention': SeqSelfAttention})
# model = load_model(model_save_path+'_all.h5',custom_objects={'bin_acc': bin_acc,'RemoveMask':RemoveMask,'RestoreMask':RestoreMask})#, 'SeqSelfAttention': SeqSelfAttention}) #如果上面的load失敗的話，可以用此
model.load_weights(model_save_path)
model.summary()
thr = 0.51 #0.482 #設定0004.於驗證資料集中所得到最佳的threshold

In [ ]:
emb_model = Model(inputs=model.inputs,  outputs=model.layers[5].output) 
#.layers[5].中的數字可能要調整，讓emb_model最後一層剛好是最終的embedding相加(Add)
emb_model.load_weights(model_save_path,by_name=True)
emb_model.summary()

In [ ]:
# 拿出直到self attention weight輸出的layer，變成att_model。
# 接下來就可以輸入某個execution profile的api function name ID跟對應的Sent2Vec vectors來用此model predict
# 輸出的結果就會是該execution profile所對應的self attention layer z
att_model = Model(inputs=model.inputs,  outputs=model.layers[-2].output) 
att_model.load_weights(model_save_path,by_name=True)
att_model.summary()

如果想要取self-attention的z出來當profile representation而不是embedding的x' 要在這裡改:
* 直接輸出self-attention那一層當成outputs

# Representative execution pattern vector
* 一個profile所有important call invocation於向量空間中的表示法r

In [ ]:
# 要記得先手動更改以下參數!! (依據不同dataset load近來的參數名稱會不同)
def avg_train_vector(root_dir,thr,train_pid_paths=exp_pid_paths,train_emb_api=exp_api,
                     train_emb=exp_emb,loner=False):
    '''
    train_pid_paths: 每一筆profile路徑位置
    train_emb_api: api embedding vectors
    train_emb: SentVec embedding vectors
    loner: False=>train/dev/test Set ; True: Loner Set
    '''
    no_ind_rep = 0
    no_ind_rep_li = []
    tree_vec = []
    empty_hash_li = []
    problem_hash_li = []
    un_aliase_path = './data/tree-rep-profiles_o2o/EXP_rev/'
    if not os.path.exists(root_dir):
        os.makedirs(root_dir,exist_ok=True)
    for path,api,emb in tqdm(zip(train_pid_paths,train_emb_api,train_emb)): #tqdm
        if loner:
            if path not in (claim_path + unk_path):
                continue
            recent_fam = path.split('/')[-3] #fakeav_fakealeart
            recent_hash = path.split('/')[-1].split('_')[0]
            if '_' in recent_fam:
                all_path_un_aliase = glob.glob(un_aliase_path+'*/*/'+current_hash+'*')
                recent_fam = all_path_un_aliase[0].split('/')[4]
                
        else:
            recent_fam = path.split('/')[-3].split('_')[0]
        recent_tree = path.split('/')[-2]
        hash_id = path.split('/')[-1][:5]
        pid_id = path.split('/')[-1].split('_')[-1].split('.')[0][-3:]
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if loner:
            if length < 2:
                empty_hash_li.append(path)
                continue
        else:
            if length < 11:
                print('No common rep. Should not in a tree:',path)
                problem_hash_li.append(path)
                continue
        try:
            assert api[:length][-1] != 0
            assert api[:length+1][-1] == 0
        except AssertionError:
            print('Mask assertion ERR:',path)
            continue
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0)
        final_emb_e = emb_model.predict([api_e,emb_e]) #(1,length,768)
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e])
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr)
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if (sum(emb) <0) or (sum(emb)>0):
                rep_emb.append(emb)
        rep_emb_avg = np.average(rep_emb,axis=0) #把自己各api的embedding取平均，應該一定要有，不然就代表這個profile不該屬於這棵tree 
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):
                no_ind_rep+=1
                no_ind_rep_li.append(path)
                continue #沒有individual REP
        except ValueError: #正常profile
            pass
        rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)
        pickle.dump(file=open(root_dir+recent_fam+'_'+recent_tree+'_'+hash_id+'_'+pid_id+'.pkl','wb'),obj=rep_emb_avg)
    print('No ind REP#:',no_ind_rep)
    if loner:
        return no_ind_rep_li,empty_hash_li,problem_hash_li
    return no_ind_rep_li

產生train/valid/test的r，用以下，要更改root_dir、及上述function的輸入變數

In [ ]:
# train/valid/test要改三次跑三次
root_dir = './data/tree-rep-profiles_o2o/trainingProfile_REP/' #最終輸出r的資料夾，會自動創立資料夾，存放pkl，記得最後的斜線
no_ind_rep = avg_train_vector(root_dir=root_dir,thr=thr,loner=False)
with open(root_dir + 'no_rep_file.txt','w') as fp:
    for path in no_ind_rep:
        fp.write(str(path.split('/')[-3:])+'\n') #no rep的profile path紀錄檔

產生loner的r，用以下，要更改root_dir(輸出r的資料夾)、及上述function的輸入變數

In [ ]:
# loner
loner_dir = './data/tree-rep-profiles_o2o/lonerProfile_REP/'
no_ind_rep,empty_files,problem_files = avg_train_vector(root_dir=loner_dir,thr=0.482,loner=True)
with open(loner_dir + 'no_rep_file.txt','w') as fp:
    for path in no_ind_rep:
        fp.write(str(path.split('/')[-1:])+'\n') #-no rep的profile path紀錄檔
with open(loner_dir + 'empty_file.txt','w') as fp:
    for path in empty_files:
        fp.write(str(path.split('/')[-1:])+'\n') #空的或太短的profile
with open(loner_dir + 'error_file.txt','w') as fp:
    for path in problem_files:
        fp.write(str(path.split('/')[-1:])+'\n') #有問題的profile(assertion failed)

* testing set之BT與F家族歸類，建議直接改由testing set的r來直接與training set的r比較cosine similarity判斷是否歸類成功 (可參考legacy code改寫)
* loner Set之F歸類，建議改由上述loner set所得的r直接與training set的r比較cosine similarity判斷是否歸類成功 (可參考legacy code改寫)
* train set的r只拿有在DEV set family中的家族

# Legacy code
* 因為當時口試及論文時間太趕，再加上學姊當時所需要的數據與圖表較多，因此這部分的code較亂。建議改寫或重寫

### testing set profiles歸類(行為樹、惡意程式家族)
* TY論文Sec 4.3實驗分析
* 以process為單位，與training set process的r比較cosine similarity
* 輸入: testing是profile(文字型)、training是r(vector型) [建議修改為都由vector型來比較]

測試資料集之representative execution pattern準備

In [ ]:
def prepare_inf_data(ind_rep_dir): #路徑要有最後的反斜線
    ind_reps = next(os.walk(ind_rep_dir))[2]
    ind_rep_tree = []
    ind_rep_fam = []
    for i,c in enumerate(ind_reps):
        ind_rep = pickle.load(open(ind_rep_dir + c,'rb')) #每個tree的centroid pickle
        if i==0:
            ind_reps_matrix = ind_rep # 1st time
        else:
            ind_reps_matrix = np.concatenate((ind_reps_matrix,ind_rep),axis=0) #依順序合在一起 
        ind_rep_tree.append(c.split('_')[1])
        ind_rep_fam.append(c.split('_')[0].split('.')[-1])
    assert ind_reps_matrix.shape[0] == len(ind_rep_tree) == len(ind_rep_fam)
    return ind_reps_matrix,ind_rep_tree,ind_rep_fam

與testing profile同樣都是最相近(最高分)的training profile有誰

In [ ]:
def EM_index(score):
    '''
    input: cosin sim 分數
    output: 同分的index
    '''
    scores = score[0].argsort() #相似度小到大
    scores = np.flip(scores) # 相似度大到小
    tmp_score = np.around(np.max(score),decimals=4) # 取到小數點地五為四捨五入到第四位
    em_id = []
    for s in scores:
        now_score = np.round(score[0][s],decimals=4) #現在這個pairwise 的similarity
        if now_score ==tmp_score: #同分增額錄取
            em_id.append(s)
    return em_id,tmp_score

* 測試資料集BT與F歸類match rate
* 論文中Figure 4.13的視覺化圖
* 沒有r的profile分析
* mismatch的profile分析

In [ ]:
#  (實驗二的code ，找Family的時候，計算match的地方在哪裡。以及求profiles in training set的地方在哪裡)
def inf_evaluate(inf_pid_paths,inf_emb_api,inf_emb,ind_reps_matrix,ind_rep_tree,ind_rep_fam,thr=thr):
    em_up = 0
    no_ind_rep = 0
    bound_accuracy = []
    wrong_tree = {}
    wrong_fam = {}
    REP_len = []
    all_length = []
    family_match = [] # 分對家族 1 否則-1 # 僅限有REP者
    family_nmatch = [] # 分錯家族 2 否則-1
    tree_match = [] # 分對BT 3 否則-1
    tree_nmatch = [] # 分錯BT 4 否則-1
    fam_li = [] #ground truth of fam
    all_predict_fam = []
    tree_li = [] #ground truth of tree
    all_predict_tree = []
    EM_sim_score = []
    
    family_match_all = [] # 分對家族 1 否則-1
    family_nmatch_all = [] # 分錯家族 2 否則-1
    tree_match_all = [] # 分對BT 3 否則-1
    tree_nmatch_all = [] # 分錯BT 4 否則-1    
    no_rep_list= []
    for path,api,emb in tqdm(zip(inf_pid_paths,inf_emb_api,inf_emb)): #tqdm
        recent_tree = path.split('/')[-2]
        recent_fam = path.split('/')[-3].split('_')[0].split('.')[-1] #EX: shodi
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if length < 1:
            print('Empty profile:',path)
            continue
        assert api[:length][-1] != 0 #確認profile跟所load 近來的vector是一致的
        assert api[:length+1][-1] == 0
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0)
        final_emb_e = emb_model.predict([api_e,emb_e]) #(1,length,768)
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e])
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        rep_length = sum(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr)
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if (sum(emb) <0) or (sum(emb)>0):
                rep_emb.append(emb)
        rep_emb_avg = np.average(rep_emb,axis=0) #把自己各api的embedding取平均，應該一定要有，不然就代表這個profile不該屬於這棵tree 
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):
                no_ind_rep+=1
                no_rep_list.append(path)
                family_match_all.append(-1) # 分對家族 1 否則-1
                family_nmatch_all.append(2)  # 分錯家族 2 否則-1
                tree_match_all.append(-1) # 分對BT 3 否則-1
                tree_nmatch_all.append(4)
                continue #沒有individual REP
        except ValueError: #正常profile
            pass
        
        REP_len.append(rep_length)
        all_length.append(length)
        
        rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)
        score = cosine_similarity(rep_emb_avg,ind_reps_matrix)
        
        scores = score[0].argsort() #相似度小到大
        scores = np.flip(scores) # 相似度大到小
        for idx in scores:
            if ind_rep_tree[idx] == recent_tree:
                bound_accuracy.append(score[0][idx]) #最差都是幾分的時候合併
        
        idx_list,max_score = EM_index(score) # exact match的id
        EM_sim_score.append(max_score)
        em_tree = []
        for idx in idx_list:
            em_tree.append(ind_rep_tree[idx])
        em_tree = list(set(em_tree)) # exact match的tree name
        all_predict_tree.append(em_tree)
        tree_li.append(recent_tree)
        if recent_tree in em_tree: #tree match
            em_up = em_up + 1
            tree_match.append(3) #畫圖
            tree_nmatch.append(-1) # 畫圖
            tree_match_all.append(3) # 分對BT 3 否則-1
            tree_nmatch_all.append(-1)            
        else: #tree not match
            tree_nmatch.append(4)
            tree_match.append(-1)
            tree_match_all.append(-1) # 分對BT 3 否則-1
            tree_nmatch_all.append(4)         
            wrong_tree[path]=em_tree
            
        em_fam = []
        for idx in idx_list:
            em_fam.append(ind_rep_fam[idx])
        em_fam = list(set(em_fam)) # exact match的tree name
        all_predict_fam.append(em_fam)
        fam_li.append(recent_fam)
        if recent_fam in em_fam: # family match
            family_match.append(1) #畫圖
            family_nmatch.append(-1) # 畫圖
            family_match_all.append(1) # 分對家族 1 否則-1
            family_nmatch_all.append(-1)
            
        else: # family not match
            family_nmatch.append(2) #畫圖
            family_match.append(-1)      # 畫圖   
            family_match_all.append(-1) # 分對家族 1 否則-1
            family_nmatch_all.append(2)
            wrong_fam[path] = em_fam
    down = len(inf_pid_paths)
#     print('ExactMatchAccuracyScore:',em_up/down,'EM#:/all#:/NoIndREP#:',em_up,down,no_ind_rep,
#          'ExactMatchPrecision(w/o No ind REP):',em_up/(down-no_ind_rep)
#          )
    return (bound_accuracy,wrong_fam,wrong_tree,em_up/down,no_ind_rep,em_up/(down-no_ind_rep),sum(REP_len)/len(REP_len),sum(all_length)/len(all_length),
           family_match_all,family_nmatch_all,tree_match_all,tree_nmatch_all,family_match,family_nmatch,tree_match,tree_nmatch,
            fam_li,tree_li,all_predict_fam,all_predict_tree,EM_sim_score,no_rep_list) #真正正確的tree是在幾%相似度才有? 、{[正確 path]=predict list}

In [ ]:
ind_rep_dir = './data/tree-rep-profiles_o2o/profile_avg/'#training set的 r 目錄位置

matrix,tree_ans,fam_ans = prepare_inf_data(ind_rep_dir)
(temp1,test_wrong_fam,test_wrong_tree,EM_score,no_rep,EMw_score,avg_rep,avg_profile,
FM_all,FN_all,TM_all,TN_all,FM,FN,TM,TN,
 fam_truth,tree_truth,fam_predict,tree_predict,EM_score_li,no_rep_path) = inf_evaluate(test_pid_paths,test_emb_api,test_emb
                                                     ,ind_reps_matrix=matrix,ind_rep_tree=tree_ans,
                                                                         ind_rep_fam=fam_ans,thr=thr)

TY論文Sec 4.3 mismatch Family的testing profiles分析
* 原本所屬的fam name/所屬的tree ID/hash/pid/模型預測的families

In [ ]:
test_mismatch_fam_pd = pd.DataFrame(columns=['ori_fam','ori_tree','sha256','pid','predict_fam'])
for i , (path,pred) in enumerate(test_wrong_fam.items()):
    fam = path.split('/')[4].split('_')[0].split('.')[-1]
    tree = path.split('/')[5]
    sha256 = path.split('/')[6].split('_')[0]
    pid = path.split('/')[-1].split('_')[-1].split('.')[0]
    test_mismatch_fam_pd.loc[i] = [fam,tree,sha256,pid,pred]
test_mismatch_fam_pd.to_excel('./data/tree-rep-profiles_o2o/test_mismatch_fam_sha256.xlsx') #輸出到excel的路徑
test_mismatch_fam_pd

TY論文Sec 4.3 mismatch BT的testing profiles分析
* 原本所屬的fam name/所屬的tree ID/hash/pid/模型預測的tree ID

In [ ]:
test_mismatch_tree_pd = pd.DataFrame(columns=['ori_fam','ori_tree','sha256','pid','predict_tree'])
for i , (path,pred) in enumerate(test_wrong_tree.items()):
    fam = path.split('/')[4].split('_')[0].split('.')[-1]
    tree = path.split('/')[5]
    sha256 = path.split('/')[6].split('_')[0]
    pid = path.split('/')[-1].split('_')[-1].split('.')[0]
    test_mismatch_tree_pd.loc[i] = [fam,tree,sha256,pid,pred]
test_mismatch_tree_pd.to_excel('./data/tree-rep-profiles_o2o/test_mismatch_tree_sha256.xlsx') #輸出excel到此路徑'
test_mismatch_tree_pd

Sec 4.3 no representative execution pattern vector的trainig profiles分析
* family/treeID/hash/pid

In [ ]:
test_no_rep_pd = pd.DataFrame(columns=['fam','tree','sha256','pid'])
for i,path in enumerate(no_rep_path):
    fam = path.split('/')[4].split('_')[0].split('.')[-1]
    tree = path.split('/')[5]
    sha256 = path.split('/')[6].split('_')[0]
    pid = path.split('/')[-1].split('_')[-1].split('.')[0]
    test_no_rep_pd.loc[i] = [fam,tree,sha256,pid]
test_no_rep_pd.to_excel('./data/tree-rep-profiles_o2o/test_no_rep_hash.xlsx') #輸出志excel    
test_no_rep_pd

論文中Figure 4.13的視覺化圖
* 老師請你畫實驗二的family, BT的match/mismatch的code

所有的測試資料集都畫出來，包含沒有r的
* 沒有r的直接預設BT跟F都分錯

In [ ]:
# all test set draw viz
number_list = []
fam_all_list = []
for i,v in enumerate(test_pid_paths):
    number_list.append(i+1)
    fam_all_list.append(v.split('/')[-3].split('_')[0].split('.')[-1])

In [ ]:
draw_df = pd.DataFrame(data={'true_fam':fam_all_list,'numberID':number_list,'FamMatch_all':FM_all,'FamNMatch_all':FN_all
                             ,'TreeMatch_all':TM_all,'TreeNMatch_all':TN_all})
draw_df.to_excel('data/tree-rep-profiles_o2o/EM_draw_allTest.xlsx',index=False) #輸出到excel，用建議圖表
draw_df

只畫有取得r的testing profile來分析match/mismatch F or BT

In [ ]:
# w/o no REP draw
number_list = [] 
fam_list = []
for i,v in enumerate(fam_truth):
    number_list.append(i+1)
fam_truth

In [ ]:
draw_df = pd.DataFrame(data={'true_fam':fam_truth,'numberID':number_list,'FamMatch':FM,'FamNMatch':FN
                             ,'TreeMatch':TM,'TreeNMatch_all':TN})
draw_df.to_excel('data/tree-rep-profiles_o2o/EM_draw_onlywREP.xlsx',index=False)
draw_df

分析在測試資料集中歸類的F&BT之precision/recall/F1

In [ ]:
fam_truth_li = []
for fam in fam_truth:
    fam_truth_li.append([fam])
tree_truth_li = []
for tree in tree_truth:
    tree_truth_li.append([tree])
assert len(fam_truth_li) == len(tree_truth_li)

In [ ]:
mlb = MultiLabelBinarizer()
y_true = mlb.fit_transform(fam_truth_li)
y_pred = mlb.transform(fam_predict)
pre = precision_score(y_true,y_pred,average='micro')
rec = recall_score(y_true,y_pred,average='micro')
f1 = f1_score(y_true,y_pred,average='micro')
print('Family precision/recall/f1',pre,rec,f1)
mlb = MultiLabelBinarizer()
y_pred = mlb.fit_transform(tree_predict)
y_true = mlb.transform(tree_truth_li)
pre = precision_score(y_true,y_pred,average='micro')
rec = recall_score(y_true,y_pred,average='micro')
f1 = f1_score(y_true,y_pred,average='micro')
print('BT precision/recall/f1',pre,rec,f1)

有r的testing profile預測了幾個F的統計變量

In [ ]:
fam_predict_num = []
for fam in fam_predict:
    fam_predict_num.append(len(fam))
basic_statistics(fam_predict_num)

In [ ]:
# 存到excel畫分布圖
kk = dict(Counter(fam_predict_num))
argmax_fam_num_left = list(kk.keys())
argmax_fam_num_right = list(kk.values())
match_df = pd.DataFrame(data={'left':argmax_fam_num_left, 'right':argmax_fam_num_right})
match_df.to_excel('data/tree-rep-profiles_o2o/argmax_match_fam_dist.xlsx',index=False) #excel輸出路徑
match_df

有r的testing profile系統是預測了幾個BT的統計變量

In [ ]:
tree_predict_num = []
for tree in tree_predict:
    tree_predict_num.append(len(tree))
basic_statistics(tree_predict_num)

argMax score是幾分的統計變量

In [ ]:
basic_statistics(EM_score_li) #decimal=7

在 validation set family當中
* training Set profiles有無r的統計分析

In [ ]:
# 有train rep的資料夾
avg_dir = './data/tree-rep-profiles_o2o/profile_avg/' #僅存放包含於DEV set family中的training profiles之r
files = next(os.walk(avg_dir))[2]
avg_fam = []
avg_tree = []
print('在47個fam的train set中有#個profiles才有REP',len(files))
for file in files:
    fam = file.split('_')[0]
    avg_fam.append(fam)
    tree = fam +'_' +file.split('_')[1]
    avg_tree.append(tree)
avg_fam_nums = dict(Counter(avg_fam))
avg_tree_nums = dict(Counter(avg_tree))

print('在47個家族中有多少個fam完全沒有rep',len(set(train_only_fam) - set(avg_fam))) #在47個家族中有多少個fam完全沒有rep
print('在47個家族中有多少個tree完全沒有rep',len(set(train_only_tree) - set(avg_tree)))
set(train_only_fam) - set(avg_fam) , set(train_only_tree) - set(avg_tree) #哪個family沒有REP?, 哪個tree沒有REP

In [ ]:
happen_no_fam_train_only = []
for k,v in avg_fam_nums.items():
    if train_only_fam_nums[k] == v:
        continue
    happen_no_fam_train_only.append(k)
print('有發生NO ind REP的家族#:',len(set(happen_no_fam_train_only)))
happen_no_tree_train_only = []
for k,v in avg_tree_nums.items():
    if train_only_tree_nums[k] == v:
        continue
    happen_no_tree_train_only.append(k)
print('有發生NO ind REP的BT#:',len(set(happen_no_tree_train_only)))

happen_no_tree_train_only #train有發生no r的BT ID

* training Set每個family包含多少profiles

In [ ]:
train_pid_fam_num = {}
for path in train_pid_paths:
    fam_name = path.split('/')[4].split('_')[0].split('.')[-1]
    try:
        train_pid_fam_num[fam_name] = train_pid_fam_num[fam_name] + 1
    except KeyError:
        train_pid_fam_num[fam_name] = 1
train_pid_fam_num , sum(list(train_pid_fam_num.values())) # in DEV set, train set's profiles#

* training Set每個family有多少profiles有r

In [ ]:
avg_dir = './data/tree-rep-profiles_o2o/profile_avg/' # train set r dir.
train_have_rep = next(os.walk(avg_dir))[2]
train_pid_fam_haveREPnum = {}
for pkl_name in train_have_rep:
    fam = pkl_name.split('_')[0].split('.')[-1]
    try:
        train_pid_fam_haveREPnum[fam] = train_pid_fam_haveREPnum[fam] + 1
    except KeyError:
        train_pid_fam_haveREPnum[fam] = 1
train_pid_fam_haveREPnum

* training Set每個family有r的比例 (TY Thesis Fig4.16)

In [ ]:
haveREP_rate = {}
for fam,all_num in train_pid_fam_num.items():
    try:
        up = train_pid_fam_haveREPnum[fam]
    except KeyError:
        up=0
    haveREP_rate[fam] =  up/all_num
haveREP_pd = pd.DataFrame([haveREP_rate],index=['Family_Train_haveREP_ratio']).T
haveREP_pd.to_excel('data/tree-rep-profiles_o2o/Family_Train_haveREP_pid_ratio.xlsx',index=True) #儲存至excel
haveREP_pd

* training set各家族間有r的比例 (TY thesis Table4.8)

In [ ]:
haverepnum = sum(list(train_pid_fam_haveREPnum.values()))
cir_cookie = {}
for k,v in train_pid_fam_haveREPnum.items():
    cir_cookie[k] = v/haverepnum
cir_cookie_pd = pd.DataFrame([cir_cookie],index=['Family_Train_inner_ratio']).T
cir_cookie_pd.to_excel('data/tree-rep-profiles_o2o/Family_Train_allpid_ratio.xlsx',index=True) #輸出可以畫圓餅圖
cir_cookie_pd

testing set有無r之profile分布分析

In [ ]:
print('testing set中沒有rep的profile個數',len((set(no_rep_path))))

unk_ori_fam_num = []
unk_fam_have_rep_num = []
unk_ori_bt_num = []
unk_bt_have_rep_num = []
ori_fam = []
ori_bt = []
for path in no_rep_path:
    fam = path.split('/')[4].split('_')[0]
    tree = path.split('/')[5]
    ori_fam.append(fam)
    ori_bt.append(fam + '_'+tree)
    fam_have_rep = glob.glob(avg_dir+fam+'*') #have
    tree_have_rep = glob.glob(avg_dir+fam+'_'+tree+'*')  #have
    fam_all_num = train_only_fam_nums[fam] #ori
    tree_all_num = train_only_tree_nums[fam + '_'+tree] #ori
    unk_ori_fam_num.append(fam_all_num)
    unk_fam_have_rep_num.append(len(fam_have_rep))
    unk_ori_bt_num.append(tree_all_num)
    unk_bt_have_rep_num.append(len(tree_have_rep))

kk = dict(Counter(ori_fam))
fam_name = list(kk.keys())
fam_name = [x.split('.')[1] for x in fam_name]
fam_count = list(kk.values())    

* 發生在哪個family，該family發生了幾個profile no r

In [2]:

unk_fam_df = pd.DataFrame(data={'Testing no r family':fam_name, 'profiles_num':fam_count})
unk_fam_df.to_excel('data/tree-rep-profiles_o2o/argmax_unk_fam_dist.xlsx',index=False)
unk_fam_df

* 發生no rep的每一筆testing profile是在哪個family、BT當中。該family原始的profile有幾個?取得r的有幾個?該BT原始的profile有幾個?成功取得r的profile有幾個?

In [ ]:
unk_ana = pd.DataFrame(data={'Ori_Fam':ori_fam,'Ori_BT':ori_bt,'Ori_Fam_profileNum':unk_ori_fam_num,
                             'TrainFam_Have_REP_num':unk_fam_have_rep_num,'Ori_BT_profileNum':unk_ori_bt_num,
                             'TrainBT_Have_REP_num':unk_bt_have_rep_num})
unk_ana.to_excel('data/tree-rep-profiles_o2o/unk_noREP_analyze.xlsx',index=False) #儲存到excel
unk_ana #row的數量==len(no_rep_path)

* testing set中，可取得r的profile是來自train set中的哪個family、BT。該family在train set中含有多少個profile?可成功取得r的有幾個?該BT在trainSet中含有多少個profile?可成功取得r的有幾個profiles?

In [ ]:
have_rep_path = (set(test_pid_paths) - set(no_rep_path))
known_ori_fam_num = []
known_fam_have_rep_num = []
known_ori_bt_num = []
known_bt_have_rep_num = []
ori_fam = []
ori_bt = []
for path in have_rep_path:
    fam = path.split('/')[4].split('_')[0]
    tree = path.split('/')[5]
    ori_fam.append(fam)
    ori_bt.append(fam + '_'+tree)
    fam_have_rep = glob.glob(avg_dir+fam+'*') #have
    tree_have_rep = glob.glob(avg_dir+fam+'_'+tree+'*')  #have
    fam_all_num = train_only_fam_nums[fam] #ori
    tree_all_num = train_only_tree_nums[fam + '_'+tree] #ori
    known_ori_fam_num.append(fam_all_num)
    known_fam_have_rep_num.append(len(fam_have_rep))
    known_ori_bt_num.append(tree_all_num)
    known_bt_have_rep_num.append(len(tree_have_rep))

known_ana = pd.DataFrame(data={'Ori_Fam':ori_fam,'Ori_BT':ori_bt,'Ori_Fam_profileNum':known_ori_fam_num,
                             'TrainFam_Have_REP_num':known_fam_have_rep_num,'Ori_BT_profileNum':known_ori_bt_num,
                             'TrainBT_Have_REP_num':known_bt_have_rep_num})
known_ana.to_excel('data/tree-rep-profiles_o2o/known_wREP_analyze.xlsx',index=False) #輸出志excel路徑
known_ana     #row數量，代表可取得r的profiles (in testing set)

為了人工去看為什麼會沒有r: 把沒有r的要分析家族profiles複製到另外一個資料夾

In [ ]:
analyze_dest = './data/tree-rep-profiles_o2o/analyze_noREP/Loner/' #改
analyze_fam = 'allaple'
out_dir = analyze_dest+analyze_fam+'/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir,exist_ok=True)
for path in tqdm(unk_path): #改 #unk_path #no_rep_path
    if analyze_fam in path:
        profile_name = path.split('/')[-1]
        shutil.copy(path,out_dir+profile_name)

# Loner家族歸類實驗與分析
* TY thesis Sec. 4.4

In [ ]:
def statistic_hash_pids(root_dir):
    fam_dir = next(os.walk(root_dir))[1]
    all_pids= []
    all_pid_list = []
    tree_count = 0
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir + fam))[1]
        tree_count += len(tree_dir)
        for tree in tree_dir:
            in_directory = root_dir + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            hash_list = [f.split('/')[-1].split('_')[0] for f in hl_list]
            pid_list = [f.split('/')[-1] for f in hl_list]
            all_pids.extend(hash_list)
            all_pid_list.extend(pid_list)
#             print(fam,len(hash_list)) #DEBUG
    all_hash = set(all_pids)
    print('Families#:',len(fam_dir),'Samples#:',len(all_hash),',Processes#:',len(set(all_pid_list)),',Trees#:',tree_count)
    return all_hash, list(set(all_pid_list))

### effective match (topN) loner家族歸類
* 門檻要測幾個就要重複跑幾次

In [ ]:
def topN_index(score,sim_score=0.8): #改:多大相似度以上就拿進來 (effective match 門檻)
    '''
    input: cosin sim 分數
    output: topN的index
    '''
    scores = score[0].argsort() #相似度小到大
    scores = np.flip(scores) # 相似度大到小
    tmp_score = np.around(np.max(score),decimals=4) # 取道小數點地五為四捨五入到第四位
    topN_id = []
    for idx in scores:
        if score[0][idx] < sim_score:
            break
        topN_id.append(idx)
    return topN_id

In [ ]:

def loner_evaluate(inf_pid_paths,inf_emb_api,inf_emb,ind_reps_matrix, #ind_reps_matrix
                   ind_rep_fam,thr=thr,debug=True): #ind_rep_fam
    em_up = 0
    no_ind_rep_hash = []
    bound_accuracy = []
    wrong_fam = {}
    wrong_fam_em = {}
    processed_hash = {}
    processed_hash_topN = []
    i=-1
    topN_up = 0
    empty_count = 0
    
    EM_path = []
    topN_path = []
    unk_path = []
    claim_path = []
    sample_topNpredict_num = {}
    no_match_anyone = []
    
    #debug
    em_fam_ori = {}
    topN_fam_ori = {}
#     mismatch_path = [] #改成用減的 (calim_path - EM - topN)
    
    for path,api,emb in tqdm(zip(inf_pid_paths,inf_emb_api,inf_emb)): #tqdm
        i+=1
        split_flag = False
        recent_fam = path.split('/')[-3] #EX: fakeav_fakealert
        if '_' in recent_fam:
            split_flag = True
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if length < 2:
#             print('Empty profile:',path)
            empty_count += 1
            continue
        try:
            assert api[:length][-1] != 0 #確認profile跟所load 近來的vector是一致的
            assert api[:length+1][-1] == 0
        except:
#             print('===Assertion ERR:',i,path)
            continue
        recent_hash = path.split('/')[-1].split('_')[0]
        if recent_hash in processed_hash.keys():
            if processed_hash[recent_hash] == 1:
                pass
        else:
            processed_hash[recent_hash] = 0        
        
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0)
        final_emb_e = emb_model.predict([api_e,emb_e]) #(1,length,768)
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e])
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr)
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if (sum(emb) <0) or (sum(emb)>0):
                rep_emb.append(emb)
        rep_emb_avg = np.average(rep_emb,axis=0) #把自己各api的embedding取平均，應該一定要有，不然就代表這個profile不該屬於這棵tree 
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):
                no_ind_rep_hash.append(recent_hash)
                unk_path.append(path)
                continue #沒有individual REP
        except ValueError: #正常profile
            pass
        rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)
        score = cosine_similarity(rep_emb_avg,ind_reps_matrix)
        
        scores = score[0].argsort() #相似度小到大
        scores = np.flip(scores) # 相似度大到小
        for idx in scores:
            if ind_rep_fam[idx] == recent_fam:
                bound_accuracy.append(score[0][idx]) #最差都是幾分的時候合併 #by_process
        
        idx_list,temp = EM_index(score) # exact match的id
        idx_list_topN = topN_index(score)
        em_fam = []
        for idx in idx_list:
            em_fam.append(ind_rep_fam[idx])
        em_fam = list(set(em_fam)) # exact match的tree name
        topN_fam = []
        for idx in idx_list_topN:
            topN_fam.append(ind_rep_fam[idx])
        topN_fam = list(set(topN_fam))

        if len(topN_fam)>0:
            try:
#                 print(sample_topNpredict_num[recent_hash],topN_fam,recent_hash)
                sample_topNpredict_num[recent_hash].extend(topN_fam)
            except KeyError:
#                 print(topN_fam)
                sample_topNpredict_num[recent_hash] = topN_fam
        else:
#                     if (topN_fam == []) or (topN_fam is None):
            no_match_anyone.append(path)
        
        flag = False #預設沒分對類別
        fams = []
        for fam in em_fam:
            if fam in recent_fam:# recent_fam: 答案
                processed_hash[recent_hash] = 1
                EM_path.append(path)
                flag = True #分對類
                fams.append(fam)
                if split_flag and debug:
                    em_fam_ori[path] = fams
#                     print(path,fam,recent_fam) #profile path、predict、true
                if not debug:
                    break
        if not flag: #依然沒分對類別
            wrong_fam_em[path]=em_fam #by_process最接近的是誰
        if recent_hash not in processed_hash_topN:
            fams = []
            for fam in topN_fam:
                if fam in recent_fam:
                    topN_up = topN_up + 1
                    topN_path.append(path)
                    flag = True
                    fams.append(fam)
                    if split_flag and debug:
                        topN_fam_ori[path] = fams
#                         print(path,fam,recent_fam)
                    if not debug:
                        break
            processed_hash_topN.append(recent_hash)
            if not flag:
                wrong_fam[path]=topN_fam
        claim_path.append(path)
    em_up = sum(list(processed_hash.values()))
    down = len(processed_hash)
    no_ind_rep = len(list(set(no_ind_rep_hash) - set(processed_hash.keys())))#有問題
#     print('ExactMatchAccuracyScore:',em_up/down,'EM_hash#:/all_hash#:/NoIndREP_sample#:',em_up,down,no_ind_rep,
#          'ExactMatchPrecision(w/o No ind REP):',em_up/(down-no_ind_rep),
#           'TopNAccScore:',topN_up/len(processed_hash_topN),'TopNAccScore(w/o No ind REP):',topN_up/(len(processed_hash_topN)-no_ind_rep),
#           'Empty process#:',empty_count
#          )
    return (bound_accuracy,wrong_fam,processed_hash,topN_up/(len(processed_hash_topN)-no_ind_rep),
           EM_path,topN_path,unk_path,claim_path,sample_topNpredict_num,no_match_anyone,
            em_fam_ori,topN_fam_ori) #真正正確的tree是在幾%相似度才有? 、{[正確 path]=predict list}

記得要改門檻是0.8/0.9 (上面function要改，下面變數名稱要改)

In [ ]:
ind_rep_dir = './data/tree-rep-profiles_o2o/profile_avg/'#'train set r
matrix,tree_ans,fam_ans = prepare_inf_data(ind_rep_dir)
(temp1,loner_misFam_08,temp3,topN_score,
EM_path,topN_path,unk_path,claim_path,
 sample_topNpredict_num,no_match_anyone_path,
em_split_fam_ori,top08_split_fam_ori) = loner_evaluate(exp_pid_paths,exp_api,exp_emb
                                                         ,ind_reps_matrix=matrix,ind_rep_fam=fam_ans,thr=thr)


### loner set no r 分析

In [ ]:
all_hash_in_loner = list(set(temp3))
print('loner dataset hash數量:',len(all_hash_in_loner),'processes數量:',len(claim_path)+len(unk_path))

In [ ]:
match_mismatch_hash = [x.split('/')[-1].split('_')[0] for x in claim_path]
print('Samples# in topNMatch/ExactMatch/Mismatch:',len(set(match_mismatch_hash)),'pids#',len(match_mismatch_hash))
unk_hash = list(set([x.split('/')[-1].split('_')[0] for x in unk_path]))

In [ ]:
pure_no_rep_sample = []
is_child = []
is_main = []
for path in unk_path:
    hash_id = path.split('/')[-1].split('_')[0]
    pid = int(path.split('/')[-1].split('_')[1].split('.')[0])
    corr_samples = [x for x in claim_path if hash_id in x]
    if corr_samples == []:
        pure_no_rep_sample.append(path)
    else:
        corr_samples_pid = [int(x.split('/')[-1].split('_')[1].split('.')[0]) for x in corr_samples]
        if pid>min(corr_samples_pid):
            is_child.append(path)
        else:
            is_main.append(path)
pure_no_rep_sample_hash = list(set([x.split('/')[-1].split('_')[0] for x in pure_no_rep_sample]))
print('某loner完全沒有任何一個process有REP的process數量:',len(pure_no_rep_sample),'Samples數量:',len(pure_no_rep_sample_hash))
print('有其他process有REP，但有部分process沒REP且屬於child的pid數量是:',len(is_child),'沒REP是main pid的數量:',len(is_main))

Sec 4.4 lonerSet no representative execution pattern vector的samples/profiles

In [ ]:
loner_no_rep_pd = pd.DataFrame(columns=['fam','sha256','pid'])
for i,path in enumerate(pure_no_rep_sample):
    fam = path.split('/')[4].split('_')[0].split('.')[-1]
#     tree = path.split('/')[5]
    sha256 = path.split('/')[6].split('_')[0]
    pid = path.split('/')[-1].split('_')[-1].split('.')[0]
    loner_no_rep_pd.loc[i] = [fam,sha256,pid]
loner_no_rep_pd.to_excel('./data/tree-rep-profiles_o2o/loner_no_rep_hash.xlsx',index=False) #輸出志excel    
loner_no_rep_pd

loner set 各家族所含的samples#

In [ ]:

all_path = []
all_path.extend(claim_path)
all_path.extend(unk_path)

processed_sample = []
fam_hash_num = {} #每個family在loner set當中有多少samples
for path in all_path:
    current_hash = path.split('/')[-1].split('_')[0]
    current_fam = path.split('/')[4]
    if current_hash in processed_sample:
        continue
    else:
        processed_sample.append(current_hash)
    try:
        fam_hash_num[current_fam] = fam_hash_num[current_fam] + 1 
    except KeyError:
        fam_hash_num[current_fam] = 1
fam_hash_num ,len(fam_hash_num) , sum(list(fam_hash_num.values())) #分布情形, family總數, samples總數

In [ ]:
# 將分布輸出到excel
fam_unkhash_num_pd = pd.DataFrame([fam_hash_num],index=['fam_unkhash_num']).T
fam_unkhash_num_pd.to_excel('data/tree-rep-profiles_o2o/fam_unkhash_num.xlsx',index=True)
fam_unkhash_num_pd

loner set中各家族沒有r的profiles含蓋了多少個samples

In [ ]:

processed_sample = []
fam_unkhash_num = {} #每個family在loner set當中有多少unk samples
for path in pure_no_rep_sample:
    current_hash = path.split('/')[-1].split('_')[0]
    current_fam = path.split('/')[4]  
    if current_hash in processed_sample:
        continue
    else:
        processed_sample.append(current_hash)
    try:
        fam_unkhash_num[current_fam] = fam_unkhash_num[current_fam] + 1 
    except KeyError:
        fam_unkhash_num[current_fam] = 1
fam_unkhash_num ,len(fam_unkhash_num)

In [ ]:
# 將分布輸出到excel
fam_unkhash_num_pd = pd.DataFrame([fam_unkhash_num],index=['fam_unkhash_num']).T
fam_unkhash_num_pd.to_excel('data/tree-rep-profiles_o2o/fam_unkhash_num.xlsx',index=True)
fam_unkhash_num_pd

利用excel畫出每個家族的no r 比例

In [ ]:
# draw all family unk rate
unk_rate = {}
for fam,all_num in fam_hash_num.items():
    try:
        up = fam_unkhash_num[fam]
    except KeyError:
        up=0
    unk_rate[fam] = up/all_num
unk_pd = pd.DataFrame([unk_rate],index=['Unknown_rate']).T
unk_pd.to_excel('data/tree-rep-profiles_o2o/FamilyUNK_rate_loner.xlsx',index=True) #路徑自行修改
unk_pd

loner set中可成功取得r的family 分布

In [ ]:


processed_sample = []
fam_hash_woUNK_num = {} #每個family在loner set當中有多少samples是具有REP的
for path in claim_path:
    current_hash = path.split('/')[-1].split('_')[0]
    current_fam = path.split('/')[4]
    if current_hash in processed_sample:
        continue
    else:
        processed_sample.append(current_hash)
    try:
        fam_hash_woUNK_num[current_fam] = fam_hash_woUNK_num[current_fam] + 1 
    except KeyError:
        fam_hash_woUNK_num[current_fam] = 1
fam_hash_woUNK_num ,len(fam_hash_woUNK_num) , sum(list(fam_hash_woUNK_num.values()))

In [ ]:
# 儲存至excel
fam_hash_woUNK_mis_num_pd = pd.DataFrame([fam_hash_woUNK_num],index=['fam_hash_haveREP_num']).T
fam_hash_woUNK_mis_num_pd.to_excel('./data/tree-rep-profiles_o2o/loner_fam_dist.xlsx')
fam_hash_woUNK_mis_num_pd

### loner set effective match與mismatch分析
* 不同相似度門檻要反覆做不同回合

In [ ]:
EM_hash = [x.split('/')[-1].split('_')[0] for x in EM_path]
topN_hash = [x.split('/')[-1].split('_')[0] for x in topN_path]
mismatch_hash = list(set(match_mismatch_hash) - set(topN_hash) -set(EM_hash))
print('mismatch sample#:',len(mismatch_hash))
exp_dir = './data/tree-rep-profiles_o2o/EXP/' #loner Set目錄
mismatch_path = []
for hash_m in mismatch_hash:
    sample_pids = glob.glob(exp_dir+'*/*/'+hash_m+'*')
    mismatch_path.extend(sample_pids)
print('mismatch process#:',len(mismatch_path))

每個family有多少個sample是分錯的
* 注意變數名稱，不同門檻要記得更改

In [ ]:

loner08_mismatch_fam_pd = pd.DataFrame(columns=['ori_fam','sha256'])
processed_sample = []
fam_hash_woUNK_mis_num = {} #每個family有多少個mismatch sample
for i,path in enumerate(mismatch_path):
    current_hash = path.split('/')[-1].split('_')[0]
    current_fam = path.split('/')[4]
#     pred = loner_misFam_09[path]
    if current_hash in no_match_sample:
        continue    

    if current_hash in processed_sample:
        continue
    else:
        processed_sample.append(current_hash)
    try:
        fam_hash_woUNK_mis_num[current_fam] = fam_hash_woUNK_mis_num[current_fam] + 1 
    except KeyError:
        fam_hash_woUNK_mis_num[current_fam] = 1
    loner08_mismatch_fam_pd.loc[i] = [current_fam,current_hash]
fam_hash_woUNK_mis_num ,len(fam_hash_woUNK_mis_num) , sum(list(fam_hash_woUNK_mis_num.values())) #家族分布, 發生的family, 總共包含多少個mismatch sample

In [ ]:
# 儲存至excel，注意檔名與路徑
fam_hash_woUNK_mis_num_pd = pd.DataFrame([fam_hash_woUNK_mis_num],index=['fam_hash_woUNK_mis_num']).T
fam_hash_woUNK_mis_num_pd.to_excel('data/tree-rep-profiles_o2o/fam_hash_woUNK_mis_num08.xlsx',index=True)
fam_hash_woUNK_mis_num_pd

Sec 4.4 mismatch Family的sample
* 不同門檻記得要更改變數以及檔名

In [ ]:
#儲存至execel
loner08_mismatch_fam_pd.to_excel('./data/tree-rep-profiles_o2o/loner08_mismsatch_fam.xlsx',index=False)
loner08_mismatch_fam_pd

loner set中各個家族effective match比例
* 不同門檻要跑不同次

In [ ]:
# draw all family unk rate
match_rate = {}
for fam,all_num in fam_hash_woUNK_num.items():
    try:
        up = fam_hash_woUNK_mis_num[fam]
    except KeyError:
        up=0
    match_rate[fam] = 1 - up/all_num
acc_pd = pd.DataFrame([match_rate],index=['Family_Match_rate']).T
acc_pd.to_excel('data/tree-rep-profiles_o2o/FamilyMatch08_rate_loner.xlsx',index=True) #儲存至excel
acc_pd

undecided sample發生在哪些家族? 跟train set r的相似度都低於門檻

In [ ]:
# all predicted family below  threshold (sample)
print(len(no_match_anyone_path)) #PID
no_match_anyone_path_hash = list(set([x.split('/')[-1].split('_')[0] for x in no_match_anyone_path]))
no_match_sample = list(set(no_match_anyone_path_hash) - set(sample_topNpredict_num.keys()))
print('有多少個sample最近的family的相似度都低於topN thr:',len(no_match_sample))
fam_no_match = []
processed_sample = []

for path in no_match_anyone_path:  
    current_hash = path.split('/')[-1].split('_')[0]
    if current_hash in processed_sample:
        continue
    else:
        processed_sample.append(current_hash)  
    current_fam = path.split('/')[4]  
    if current_hash in no_match_sample:
        fam_no_match.append(current_fam)
print(len(fam_no_match))
fam_no_match_stat = dict(Counter(fam_no_match))
fam_no_match_stat

loner set每個sample預測了幾個F的統計資料
* 不同相似度門檻要分開做不同次

In [ ]:
hash_topNpredict_num = []
for key, li in sample_topNpredict_num.items():
    hash_topNpredict_num.append(len(set(li)))
print('共有多少個samples至少預測一個family:',len(hash_topNpredict_num))
basic_statistics(hash_topNpredict_num)


In [ ]:
# 將分布存到excel畫圖
sample_topNpredict_num_df = dict(Counter(hash_topNpredict_num))
sample_topNpredict_num_df = pd.DataFrame([sample_topNpredict_num_df],index=['topN_fam_predict_num']).T
sample_topNpredict_num_df.to_excel('data/tree-rep-profiles_o2o/topN08_predictFam_num_loner.xlsx',index=True)
sample_topNpredict_num_df

每個sample是預測了哪些family出來?
* 以0.9 相似度門檻為例

In [ ]:
sample_belongFam09_pd = pd.DataFrame(columns=['sha256','predict_fam'])
all_values_bySample = []
for i,(key, li) in enumerate(sample_topNpredict_num.items()):
    value = list(set(li))
    all_values_bySample.extend(value)
    sample_belongFam09_pd.loc[i] = [key,value]
sample_belongFam09_pd.to_excel('./data/tree-rep-profiles_o2o/sample_belongFam09.xlsx',index=False)
sample_belongFam09_pd

所有的家族中每個家族共被系統預測了幾次 (sample很靠近的是來自哪些family)
* 不同門檻要跑不同次

In [ ]:
fam_loner_sampleNum_df = dict(Counter(all_values_bySample))
fam_loner_sampleNum_df = pd.DataFrame([fam_loner_sampleNum_df],index=['loner_sampples_num']).T
fam_loner_sampleNum_df.to_excel('./data/tree-rep-profiles_o2o/fam_loner_sampleNum09.xlsx') #儲存至excel
fam_loner_sampleNum_df

# Representative execution pattern visualize
* TY thesis Sec. 4.5
* by umap 2D
* tool: http://projector.tensorflow.org/

#### 畫出某個family的BT分布
* train & test
* 如果想用在實驗二畫unknown sample應該怎麼做: 求出train的embedding vectors、求出test的embedding vector。並把vector中每個值用\t分開。metadata的部分則可以放自己想要看的標籤。詳細可參考 http://projector.tensorflow.org/ 的load按鈕裡面的教學

In [ ]:
fam_name = '3.allaple'
train_dir =  './data/tree-rep-profiles_o2o/profile_avg/'+fam_name+'*' #avg rep的位置
train_avg_pickles = glob.glob(train_dir)
# test_dir = './data/tree-rep-profiles_o2o/TEST/'+fam_name+'_0.8/'
# thr =0.51 #0.482

In [ ]:
train_avg_emb = []
train_tree_name = ['Tree\tType\tID']
for i,avg_emb_path in enumerate(train_avg_pickles):
    tree_name = avg_emb_path.split('/')[-1].split('_')[1]
    emb = pickle.load(open(avg_emb_path,'rb'))[0]
    train_avg_emb.append(emb)
    train_tree_name.append(tree_name+'\t'+'Train'+'\t'+str(i))
i

In [ ]:
def avg_target_vector(target_pid_paths,target_emb_api,target_emb,thr):
    '''
    計算profile的r
    '''
    no_ind_rep = 0
    tree_vec = []
    target_avg_emb = []
    target_tree_name = []
    print(thr)
    i=392 #接續上面i的ID

    for path,api,emb in tqdm(zip(target_pid_paths,target_emb_api,target_emb)): #tqdm

        recent_tree = path.split('/')[-2]

        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if length < 2:
            print('No common rep. Should not in a tree:',path)
            continue
        assert api[:length][-1] != 0
        assert api[:length+1][-1] == 0
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0) 
        final_emb_e = emb_model.predict([api_e,emb_e]) #(1,length,768) ##api_e,emb_e
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e]) ##api_e,emb_e
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr)
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if (sum(emb) <0) or (sum(emb)>0):
                rep_emb.append(emb)
        rep_emb_avg = np.average(rep_emb,axis=0) #把自己各api的embedding取平均，應該一定要有，不然就代表這個profile不該屬於這棵tree 
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):
                no_ind_rep+=1
                continue #沒有individual REP
        except ValueError: #正常profile
            pass
        target_avg_emb.append(rep_emb_avg)
        target_tree_name.append(recent_tree+'\t'+'Test'+'\t'+str(i))
        i=i+1
#         rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)
#         pickle.dump(file=open(root_dir+recent_fam+'_'+recent_tree+'_'+hash_id+'_'+pid_id+'.pkl','wb'),obj=rep_emb_avg)
    print('No ind REP#:',no_ind_rep)
    return target_avg_emb,target_tree_name

In [ ]:
target_id = []
target_pid_paths = []
for i,path in enumerate(test_pid_paths):
    if fam_name in path:
        target_id.append(i)
        target_pid_paths.append(path)
target_emb_api = test_emb_api[target_id]
target_emb = test_emb[target_id]
assert len(target_id) == len(target_emb_api) == len(target_emb) == len(target_pid_paths)
print(target_emb.shape)

target_avg_emb,target_tree_name = avg_target_vector(target_pid_paths,target_emb_api,target_emb,thr)
assert len(target_avg_emb) == len(target_tree_name)

all_avg_emb = train_avg_emb
all_tree_name = train_tree_name
print(len(all_avg_emb),len(all_tree_name))
all_avg_emb.extend(target_avg_emb)
all_tree_name.extend(target_tree_name)
len(all_avg_emb),len(all_tree_name)

輸出要視覺化的r vectors與metadata
* 兩個檔案的輸出路徑記得修改

In [ ]:
all_avg_df = pd.DataFrame(np.array(all_avg_emb))
all_avg_df.to_csv('./data/tree-rep-profiles_o2o/allaple_emb.tsv',sep='\t',index=False,header=False)
with open ("./data/tree-rep-profiles_o2o/allaple_meta.tsv","w")as fp:
    for line in all_tree_name:
        fp.write(line+"\n")
all_avg_df

#### 畫出train Set中有取得r的家族之所有profiles之r
* train only、實驗三中的visualization的圖從一開始怎麼畫

In [ ]:
train_dir =  './data/tree-rep-profiles_o2o/profile_avg/'+'*' #train set r 的位置
train_avg_pickles = glob.glob(train_dir)
# thr = 0.51 #0.482

In [ ]:
train_avg_emb = []
train_tree_name = ['Family\tTree\tType']
for i,avg_emb_path in enumerate(train_avg_pickles):
    fam_name = avg_emb_path.split('/')[-1].split('_')[0]
    tree_name = avg_emb_path.split('/')[-1].split('_')[1]
    emb = pickle.load(open(avg_emb_path,'rb'))[0]
    train_avg_emb.append(emb)
    train_tree_name.append(fam_name+'\t'+tree_name+'\t'+'Train')
i

輸出train set中所有r的vectors及metadata labels
* 輸出位置請自行修改

In [ ]:
assert len(train_avg_emb) + 1 == len(train_tree_name)
fam_avg_df = pd.DataFrame(np.array(train_avg_emb))
fam_avg_df.to_csv('./data/tree-rep-profiles_o2o/47fam_emb.tsv',sep='\t',index=False,header=False)
with open ("./data/tree-rep-profiles_o2o/47fam_meta.tsv","w")as fp:
    for line in train_tree_name:
        fp.write(line+"\n")
fam_avg_df

### 畫出loner set中某個家族所包含的profiles之r

In [ ]:
fam_target = 'allaple'
train_dir =  './data/tree-rep-profiles_o2o/profile_avg/'+'*' #avg rep的位置
train_avg_pickles = glob.glob(train_dir)
# thr = 0.51 #0.482

In [ ]:
train_avg_emb = []
train_tree_name = ['Family\tTree\tType']
for i,avg_emb_path in enumerate(train_avg_pickles):
    fam_name = avg_emb_path.split('/')[-1].split('_')[0].split('.')[1]
    tree_name = fam_name+'_'+avg_emb_path.split('/')[-1].split('_')[1]
    emb = pickle.load(open(avg_emb_path,'rb'))[0]
    train_avg_emb.append(emb)
    train_tree_name.append(fam_name+'\t'+tree_name+'\t'+'Train')
i

In [ ]:
def avg_target_vector_fam(target_pid_paths,target_emb_api,target_emb,thr):
    no_ind_rep = 0
    tree_vec = []
    target_avg_emb = []
    target_fam_name = []
    print(thr)
    i=392 #
    for path,api,emb in tqdm(zip(target_pid_paths,target_emb_api,target_emb)): #tqdm
        recent_fam = path.split('/')[-3].split('_')[0]
        recent_tree = recent_fam + '_' + path.split('/')[-2]
#         hash_id = path.split('/')[-1][:5]
#         pid_id = path.split('/')[-1].split('_')[-1].split('.')[0][-3:]
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if length < 2:
            print('No common rep. Should not in a tree:',path)
            continue
        assert api[:length][-1] != 0
        assert api[:length+1][-1] == 0
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0) 
        final_emb_e = emb_model.predict([api_e,emb_e]) #(1,length,768) ##api_e,emb_e
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e]) ##api_e,emb_e
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr)
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if (sum(emb) <0) or (sum(emb)>0):
                rep_emb.append(emb)
        rep_emb_avg = np.average(rep_emb,axis=0) #把自己各api的embedding取平均，應該一定要有，不然就代表這個profile不該屬於這棵tree 
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):
                no_ind_rep+=1
                continue #沒有individual REP
        except ValueError: #正常profile
            pass
        target_avg_emb.append(rep_emb_avg)
        target_fam_name.append(recent_fam+'\t'+recent_tree+'\t'+'Loner')
#         i=i+1
#         rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)
#         pickle.dump(file=open(root_dir+recent_fam+'_'+recent_tree+'_'+hash_id+'_'+pid_id+'.pkl','wb'),obj=rep_emb_avg)
    print('No ind REP#:',no_ind_rep)
    return target_avg_emb,target_fam_name

In [ ]:
target_id = []
target_pid_paths = []
for i,path in enumerate(exp_pid_paths):
    if (fam_target in path) and (path in topN_path): #topN_path要先跑完loner分析
        target_id.append(i)
        target_pid_paths.append(path)
target_emb_api = exp_api[target_id]
target_emb = exp_emb[target_id]
assert len(target_id) == len(target_emb_api) == len(target_emb) == len(target_pid_paths)
print(target_emb.shape)

target_avg_emb,target_fam_name = avg_target_vector_fam(target_pid_paths,target_emb_api,target_emb,thr)
assert len(target_avg_emb) == len(target_fam_name)

all_avg_emb = []
all_avg_emb.extend(train_avg_emb)
all_fam_name = []
all_fam_name.extend(train_tree_name)
assert len(all_avg_emb)+1 == len(all_fam_name)
all_avg_emb.extend(target_avg_emb)
all_fam_name.extend(target_fam_name)
assert len(all_avg_emb)+1 == len(all_fam_name)

輸出所需之r vector與metadata label
* 記得修改輸出的兩個路徑

In [ ]:
fam_avg_df = pd.DataFrame(np.array(all_avg_emb))
fam_avg_df.to_csv('./data/tree-rep-profiles_o2o/allapleLoner_fam_emb2.tsv',sep='\t',index=False,header=False)
with open ("./data/tree-rep-profiles_o2o/allapleLoner_fam_emb_meta2.tsv","w")as fp:
    for line in all_fam_name:
        fp.write(line+"\n")
fam_avg_df

建議使用tool當中的UMAP降維視覺化，設定neighbor為5，2D

***

# Appendix Spplementary
* 與任務主線無關，只是用來測試的小方法
* Deprecated code. Need lots of modifications

## kNN classification (loner family)
* 依照loner所靠近的k個training r來投票決定歸屬哪一類 (然而因為通常都是一大群涵蓋近來，所以最多的不會是自己家族的，而且自己家族的樹量可能training太少所以無法投票成功)(data imbalance不可用)


對每一個family先進行編碼

In [ ]:
loner_dir = './data/tree-rep-profiles_o2o/EXP/' # training Set r
fam_exp = next(os.walk(loner_dir))[1]
encode_fam = {}
for i,v in enumerate(fam_exp):
    encode_fam[v] = i
pickle.dump(file=open('data/tree-rep-profiles_o2o/encode_fam.pkl','wb'),obj=encode_fam) #備份家族編碼順序
encode_fam

將 training r 轉換為kNN可用的訓練資料集向量格式

In [ ]:
# train data prepare
ind_rep_dir = './data/tree-rep-profiles_o2o/profile_avg/' #要有最後的反斜線
ind_reps = next(os.walk(ind_rep_dir))[2]
kNN_train_Y = []
for i,c in enumerate(ind_reps):
    ind_rep = pickle.load(open(ind_rep_dir + c,'rb')) #每個tree的centroid pickle
    fam_name = c.split('_')[0].split('.')[-1]
    for j,v in enumerate(list(encode_fam.keys())):
        if fam_name in v:
            kNN_train_Y.append(j)
            break
    if i==0:
        kNN_train_X = ind_rep # 1st time
    else:
        kNN_train_X = np.concatenate((kNN_train_X,ind_rep),axis=0) #依順序合在一起 
assert kNN_train_X.shape[0] == len(kNN_train_Y)

轉換驗證資料集為knn可用的格式

In [ ]:
def data_prepare(inf_pid_paths,inf_emb_api,inf_emb,encode_fam=encode_fam,loner_dir=False):
    em_up = 0
    no_ind_rep = 0
    inf_X = []
    inf_Y = []
    for path,api,emb in tqdm(zip(inf_pid_paths,inf_emb_api,inf_emb)):
        if loner_dir:
            fam_name = path.split('/')[-3]
        else:
            fam_name = path.split('/')[-3].split('_')[0].split('.')[-1]
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if length < 1:
            print('Empty profile:',path)
            continue
        try:
            assert api[:length][-1] != 0 #確認profile跟所load 近來的vector是一致的
            assert api[:length+1][-1] == 0
        except:
            print('===Assertion ERR:',i,path)
            continue
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0)
        final_emb_e = emb_model.predict([api_e,emb_e]) #(1,length,768)
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e])
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr)
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if (sum(emb) <0) or (sum(emb)>0):
                rep_emb.append(emb)
        rep_emb_avg = np.average(rep_emb,axis=0)
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):
#                 no_ind_rep_hash.append(recent_hash)
                continue #沒有individual REP
        except ValueError: #正常profile
            pass
        inf_X.append(rep_emb_avg)
        for j,v in enumerate(list(encode_fam.keys())):
            if fam_name in v:
                inf_Y.append(j)
                break        
    assert len(inf_X) == len(inf_Y)
    return np.array(inf_X),inf_Y

In [ ]:
valid_X , valid_Y = data_prepare(valid_pid_paths,valid_emb_api,valid_emb)


利用驗證資料集決定k要選多少

In [ ]:
for i in [3,5,7,9,11,13,15,17,19,21,23,25,27,29,31]:
    neigh = KNeighborsClassifier(n_neighbors=i,n_jobs=30,weights='distance') #,weights='distance'
    neigh.fit(kNN_train_X, kNN_train_Y) 
    score = neigh.score(valid_X,valid_Y)
    print('k=',i,',acc=',score) #選acc最高的k做為最終要使用的k

檢驗loner set最終的acc效能

In [ ]:
exp_X , exp_Y = data_prepare(exp_pid_paths,exp_api,exp_emb) #轉換為knn格式向量
neigh = KNeighborsClassifier(n_neighbors=1311,n_jobs=30) #,weights='distance'
neigh.fit(kNN_train_X, kNN_train_Y) 
score = neigh.score(exp_X,exp_Y)
ans = neigh.predict(exp_X)
print('最終分類準確率:',score)
ans #每一個loner predict的家族編碼

### BT centroid分類法
* 算出train set每個BT中所有profiles r的centroid
* 看loner與哪個centroid最靠近，在來看是否為原所屬家族的BT，判斷是否歸類成功
* 目前是用euclidean dist.比較

先計算每個train BT的centroid並存到root_dir中

In [ ]:
fam_name = None
tree_name = None
tree_vec = []
root_dir = './data/tree-rep-profiles_o2o/centroids/' 
if not os.path.exists(root_dir):
    os.makedirs(root_dir,exist_ok=True)
for path,api,emb in tqdm(zip(train_pid_paths,train_emb_api,train_emb)):
    recent_fam = path.split('/')[-3].split('_')[0]
    recent_tree = path.split('/')[-2]
    with open(path,encoding='ISO 8859-1') as f: #X2
        lines = f.read()
    lines = re.sub(r'[^\x00-\x7F]+','', lines)
    lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
    lines = re.sub('=\\n','',lines)
    lines = re.sub('y\\n','',lines)
    lines = lines.splitlines()
    length = len(lines)
    if length < 11:
        print('No common rep. Should not in a tree:',path)
        continue
#     print(len(lines))
    assert api[:length][-1] != 0
    assert api[:length+1][-1] == 0
    api_e = np.expand_dims(api, axis=0)
    emb_e = np.expand_dims(emb, axis=0)
    final_emb_e = emb_model.predict([api_e,emb_e]) #(1,length,768)
    final_emb = final_emb_e[0][:length] #(length,768)
    byte_rep_e = model.predict([api_e,emb_e])
    byte_rep = byte_rep_e[0][:length] # (length,1)
    byte_rep_thr = []
    for num in byte_rep:
        if num[0] < thr:
            byte_rep_thr.append(0)
        else:
            byte_rep_thr.append(1) # high than threshold, keep it
    byte_rep_thr = np.array(byte_rep_thr)
    byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
    mul_emb = np.multiply(final_emb,byte_rep_thr)
    rep_emb = [] #超過threshold的emb才留下來
    for emb in mul_emb:
        if (sum(emb) <0) or (sum(emb)>0):
            rep_emb.append(emb)
    rep_emb_avg = np.average(rep_emb,axis=0) #把自己各api的embedding取平均，應該一定要有，不然就代表這個profile不該屬於這棵tree
    if recent_fam==fam_name and recent_tree==tree_name:
        tree_vec.append(rep_emb_avg) #如果同tree同fam就把rep_emb加進去list
    else:
        if tree_vec:
            fam_tree_c = np.average(tree_vec,axis=0) # 求出整個tree的centroid
            pickle.dump(file=open(root_dir+fam_name+'_'+tree_name+'.pkl','wb'),obj=fam_tree_c)
            tree_vec = []
        fam_name = recent_fam
        tree_name = recent_tree
        if not tree_vec: # if tree vector list is empty
            tree_vec.append(rep_emb_avg)
#     break
#     print(path,api,emb)
#     break

#### training set 歸類
* calculate euclidean distance
* check if match right tree/fam
* top1/top3/top5

建議做:

* 同sample不同pid的歸類，同分的歸類 (增額錄取)
* topN dis<0.2以下的歸類 (topN effective match)
* family改成是最近的3/5個family (同family就往下推移)
* 把training沒有此tree的DEV pid拿掉

In [ ]:
centroid_dir = './data/tree-rep-profiles_o2o/centroids/' #要有最後的反斜線。訓練資料集各BT的centroid目錄
def inf_evaluate(inf_pid_paths,inf_emb_api,inf_emb,centroid_dir=centroid_dir):
    centroids = next(os.walk(centroid_dir))[2]
    centroid_matrix = None
    c_fam = []
    c_tree = []
    err_centroid = []
    top1_fam_up = 0
    top1_tree_up = 0
    top3_fam_up = 0
    top3_tree_up = 0
    top5_fam_up = 0
    top5_tree_up = 0
    wrong_fam = {}
    wrong_tree = {}
    no_ind_rep = 0
    for i,c in enumerate(centroids):
        tree_rep = pickle.load(open(centroid_dir + c,'rb')) #每個tree的centroid pickle
        tree_rep = np.expand_dims(tree_rep,axis=0)
        try:
            chk = sum(sum(tree_rep))
        except TypeError:
            err_centroid.append(c)
            continue
        if (not chk < 0) and (not chk>0):
            err_centroid.append(c)
            continue
        if i >0:
            centroid_matrix = np.concatenate((centroid_matrix,tree_rep),axis=0) #依順序合在一起       
        if i==0:
            centroid_matrix = tree_rep # 1st time
        c_fam.append(c.split('_')[0]) # centroid的fam name
        c_tree.append(c.split('_')[-1].split('.')[0]) # centroid的tree name
    assert centroid_matrix.shape[0] == len(c_fam) == len(c_tree)
    assert len(inf_pid_paths) == len(inf_emb_api) == len(inf_emb)
    for path,api,emb in tqdm(zip(inf_pid_paths,inf_emb_api,inf_emb)):
        recent_fam = path.split('/')[-3].split('_')[0]
        recent_tree = path.split('/')[-2]
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if length < 1:
            print('Empty profile:',path)
            continue
        assert api[:length][-1] != 0 #確認profile跟所load 近來的vector是一致的
        assert api[:length+1][-1] == 0
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0)
        final_emb_e = emb_model.predict([api_e,emb_e])
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e])
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr) #把embediing跟超過threshold的byterep相乘  
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if sum(emb) !=0: #跟0鄉城都會=0
                rep_emb.append(emb)       
        rep_emb_avg = np.average(rep_emb,axis=0)
        rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):
                no_ind_rep+=1
                continue #沒有individual REP
        except ValueError: #正常profile
            pass
        dist = euclidean_distances(rep_emb_avg,centroid_matrix) #取出與各rep距離，求min的idx對答案
        min_dist = np.min(dist) #top1
        min_idx = np.argmin(dist) #top1
        dist5 = dist[0].argsort()[:5] #top3
        if c_fam[min_idx] == recent_fam:
            top1_fam_up = top1_fam_up+1
        else:
            wrong_fam[path] = c_fam[min_idx]
        if (c_fam[min_idx] == recent_fam) and (c_tree[min_idx] == recent_tree):
            top1_tree_up = top1_tree_up+1
        else:
            wrong_tree[path] = c_fam[min_idx]+'_'+ c_tree[min_idx]
        if recent_fam in [c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]]]:
            top3_fam_up = top3_fam_up+1
        if (recent_tree in [c_tree[dist5[0]],c_tree[dist5[1]],c_tree[dist5[2]]]) and (recent_fam in[c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]]]):
            top3_tree_up = top3_tree_up+1
        if recent_fam in [c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]],c_fam[dist5[3]],c_fam[dist5[4]]]:
            top5_fam_up = top5_fam_up+1
        if (recent_fam in [c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]],c_fam[dist5[3]],c_fam[dist5[4]]]) and (recent_tree in [c_tree[dist5[0]],c_tree[dist5[1]],c_tree[dist5[2]],c_tree[dist5[3]],c_tree[dist5[4]]]):
            top5_tree_up = top5_tree_up + 1
    down = len(inf_pid_paths)
    print('Top1 Fam score:',top1_fam_up/down,'Top1 Tree score:',top1_tree_up/down,
         'Top3 Fam score:',top3_fam_up/down,'Top3 Tree score:',top3_tree_up/down,
          'Top5 Fam score:',top5_fam_up/down,'Top5 Tree score:', top5_tree_up/down,
          'No ind rep#:',no_ind_rep
         )
    return wrong_fam,wrong_tree
#         for c in centroids:
#             tree_rep = pickle.load(open(centroid_dir + c,'rb'))


In [ ]:
temp = inf_evaluate(test_pid_paths,test_emb_api,test_emb)

#### loner set 家族歸類

In [ ]:
centroid_dir = './data/tree-rep-profiles_o2o/centroids/' #要有最後的反斜線
def loner_evaluate(lon_pid_paths,lon_emb_api,lon_emb,centroid_dir=centroid_dir):
    centroids = next(os.walk(centroid_dir))[2]
    centroid_matrix = None
    c_fam = []
    c_tree = []
    err_centroid = []
    top1_fam_up = 0
#     top1_tree_up = 0
    top3_fam_up = 0
#     top3_tree_up = 0
    top5_fam_up = 0
#     top5_tree_up = 0
    wrong_fam = {}
#     wrong_tree = {}
    no_ind_rep = 0
    for i,c in enumerate(centroids):
        tree_rep = pickle.load(open(centroid_dir + c,'rb')) #每個tree的centroid pickle
        tree_rep = np.expand_dims(tree_rep,axis=0)
        try:
            chk = sum(sum(tree_rep))
        except TypeError:
            err_centroid.append(c)
            continue
        if (not chk < 0) and (not chk>0):
            err_centroid.append(c)
            continue
        if i >0:
            centroid_matrix = np.concatenate((centroid_matrix,tree_rep),axis=0) #依順序合在一起       
        if i==0:
            centroid_matrix = tree_rep # 1st time
        c_fam.append(c.split('_')[0]) # centroid的fam name
#         c_tree.append(c.split('_')[-1].split('.')[0]) # centroid的tree name
    assert centroid_matrix.shape[0] == len(c_fam) #== len(c_tree)
    assert len(lon_pid_paths) == len(lon_emb_api) == len(lon_emb)
    for path,api,emb in tqdm(zip(lon_pid_paths,lon_emb_api,lon_emb)):
        recent_fam = path.split('/')[-3].split('_')[0]
#         recent_tree = path.split('/')[-2]
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if length < 1:
            print('Empty profile:',path)
            continue
        assert api[:length][-1] != 0 #確認profile跟所load 近來的vector是一致的
        assert api[:length+1][-1] == 0
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0)
        final_emb_e = emb_model.predict([api_e,emb_e])
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e])
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr) #把embediing跟超過threshold的byterep相乘  
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if sum(emb) !=0: #跟0鄉城都會=0
                rep_emb.append(emb)       
        rep_emb_avg = np.average(rep_emb,axis=0)
        rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):#沒有individual REP
                no_ind_rep+=1
                continue 
        except ValueError: #正常profile
            pass
        dist = euclidean_distances(rep_emb_avg,centroid_matrix) #取出與各rep距離，求min的idx對答案
        min_dist = np.min(dist) #top1
        min_idx = np.argmin(dist) #top1
        dist5 = dist[0].argsort()[:5] #top3
        if c_fam[min_idx] == recent_fam:
            top1_fam_up = top1_fam_up+1
        else:
            wrong_fam[path] = c_fam[min_idx]
#         if (c_fam[min_idx] == recent_fam) and (c_tree[min_idx] == recent_tree):
#             top1_tree_up = top1_tree_up+1
#         else:
#             wrong_tree[path] = c_fam[min_idx]+'_'+ c_tree[min_idx]
        if recent_fam in [c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]]]:
            top3_fam_up = top3_fam_up+1
#         if (recent_tree in [c_tree[dist5[0]],c_tree[dist5[1]],c_tree[dist5[2]]]) and (recent_fam in[c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]]]):
#             top3_tree_up = top3_tree_up+1
        if recent_fam in [c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]],c_fam[dist5[3]],c_fam[dist5[4]]]:
            top5_fam_up = top5_fam_up+1
#         if (recent_fam in [c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]],c_fam[dist5[3]],c_fam[dist5[4]]]) and (recent_tree in [c_tree[dist5[0]],c_tree[dist5[1]],c_tree[dist5[2]],c_tree[dist5[3]],c_tree[dist5[4]]]):
#             top5_tree_up = top5_tree_up + 1
    down = len(lon_pid_paths)
    print('Top1 Fam score:',top1_fam_up/down,#'Top1 Tree score:',top1_tree_up/down,
         'Top3 Fam score:',top3_fam_up/down,#'Top3 Tree score:',top3_tree_up/down,
          'Top5 Fam score:',top5_fam_up/down,#'Top5 Tree score:', top5_tree_up/down,
          'No ind rep#:',no_ind_rep
         )
    return wrong_fam
#         for c in centroids:
#             tree_rep = pickle.load(open(centroid_dir + c,'rb'))


In [ ]:
temp = loner_evaluate(exp_pid_paths,exp_api,exp_emb)